In [1]:
import oceanspy as ospy
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection
import matplotlib.colors as mcolors
from tqdm import tqdm
from matplotlib.animation import FuncAnimation, FFMpegFileWriter

In [8]:
df = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/Background_var.nc')
depth_Hor_res = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/Depth_res.nc')
area = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/rA.nc')
verti_res = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/drF.nc')

In [14]:
df

<xarray.Dataset>
Dimensions:    (Z: 216, Zp1: 217, Zu: 216, Zl: 216, X: 765, Y: 628, Xp1: 766,
                Yp1: 629, time: 12, time_midp: 11)
Coordinates: (12/18)
  * Z          (Z) float64 -1.0 -3.5 -7.0 ... -3.904e+03 -3.922e+03 -3.942e+03
  * Zp1        (Zp1) float64 0.0 -2.0 -5.0 ... -3.913e+03 -3.932e+03 -3.951e+03
  * Zu         (Zu) float64 -2.0 -5.0 -9.0 ... -3.913e+03 -3.932e+03 -3.951e+03
  * Zl         (Zl) float64 0.0 -2.0 -5.0 ... -3.894e+03 -3.913e+03 -3.932e+03
  * X          (X) float64 -29.92 -29.84 -29.76 -29.67 ... 11.72 11.82 11.92
  * Y          (Y) float64 65.01 65.03 65.04 65.06 ... 78.67 78.75 78.83 78.92
    ...         ...
  * time       (time) datetime64[ns] 2017-09-01 2017-10-01 ... 2018-08-01
    YU         (Y, Xp1) float64 ...
    XU         (Y, Xp1) float64 ...
    YV         (Yp1, X) float64 ...
    XV         (Yp1, X) float64 ...
  * time_midp  (time_midp) datetime64[ns] 2017-09-16 ... 2018-07-16T12:00:00
Data variables:
    U          (time, Z, Y, Xp1) float64 ...
    V          (time, Z, Yp1, X) float64 ...
    Temp       (time, Z, Y, X) float64 ...
    S          (time, Z, Y, X) float64 ...
    Eta        (time, Y, X) float64 ...
    MXLDEPTH   (time, Y, X) float64 ...
    SIarea     (time, Y, X) float64 ...
    SIheff     (time, Y, X) float64 ...
Attributes: (12/24)
    MITgcm_version:          checkpoint67l
    build_user:              malmans2@jhu.edu
    build_host:              compute0673
    build_date:              Fri Nov  8 01:36:45 EST 2019
    MITgcm_URL:              http://mitgcm.org
    MITgcm_tag_id:           
    ...                      ...
    OceanSpy_parameters:     {'rSphere': 6371.0, 'eq_state': 'mdjwf', 'rho0':...
    OceanSpy_name:           IGPyearlong
    OceanSpy_description:    High-resolution numerical simulation carried out...
    OceanSpy_projection:     Mercator(**{})
    OceanSpy_grid_coords:    {'Y': {'Y': None, 'Yp1': 0.5}, 'X': {'X': None, ...
    OceanSpy_grid_periodic:  []

In [15]:
U = df.U
V = df.V

# Create the target grid (X and Y) based on the midpoints of the staggered grid
X = df['X']
Y = df['Y']

# Interpolating U to the target grid (X, Y)
U_interp = U.interp(Xp1=X, Y=Y, method='linear')

# Interpolating V to the target grid (X, Y)
V_interp = V.interp(X=X, Yp1=Y, method='linear')

# Drop the existing Y coordinate in V_interp to avoid conflicts
U_interp = U_interp.drop_vars('X')
V_interp = V_interp.drop_vars('Y')

# Rename the dimensions and coordinates
U_interp = U_interp.rename({'Xp1': 'X'})
V_interp = V_interp.rename({'Yp1': 'Y'})

# Ensure the coordinates are updated correctly
U_interp = U_interp.assign_coords(X=X, Y=Y)
V_interp = V_interp.assign_coords(X=X, Y=Y)

display(U_interp)

MemoryError: Unable to allocate 9.28 GiB for an array with shape (765, 1627776) and data type float64

# Compute transport in each gridcell for a monthly mean state
$Transport=\sum_z^{Z}(V*Area_V)$, $Area_V=dxF*drF$, $Area_U=dyF*drF$

In [11]:
U_transport = df.U[0]*depth_Hor_res.dyF*verti_res.drF
V_transport = df.V[0]*depth_Hor_res.dxF*verti_res.drF

MemoryError: Unable to allocate 743. GiB for an array with shape (216, 628, 766, 960) and data type float64